<div class="alert alert-info">
    <h1> About these notebooks </h1>
    <p> If you opened this notebook in Binder, it is running on a server that was launched <b>just</b> for you. Your changes will be reset once server restarts due to inactivity, so don't rely on it for anything you want to last. Likewise, feel free to try and tweak everything you want, since you won't affect the original repository. And <b>if you have never used a Jupyter noteboook: </b> for running a cell just press <tt>Shift + Enter.</tt> </p>
    <p>Enjoy!</p>
</div>

# 4.1 Conway's *Game of Life*

To get a taste about Julia's speed we are going to code Conway's *Game of Life*. But before explaining its basic rules, let's watch a cool gif of it:

![game of life gif](figures/game_of_life.gif)

So what's this "Game of Life"? Well, it is a *cellular automaton*: a 2D, infinite grid of cells ― each of which can be *alive* or *dead*―, together with rules for wether they stay in the same state or change it. In Life the next state of a cell only depends on the states of the 8 nearest neighbors, and according to [Wikipedia](https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life) the rules for every iteration are:

* Any live cell with fewer than two live neighbours dies, as if by underpopulation.
* Any live cell with two or three live neighbours lives on to the next generation.
* Any live cell with more than three live neighbours dies, as if by overpopulation.
* Any dead cell with exactly three live neighbours becomes a live cell, as if by reproduction.

We can summarize this with the following protocol. Assuming that the state of the cell $(i,j)$ is stored in `A[i,j]` (as `1` if the cell is alive and `0` if it is dead), then in each iteration:

1. Initialize an array `B` with the same shape as `A` and all zeros.
2. For each cell $(i,j)$, if either:
    * the sum of live neighbors is 3 (regardless of the state of cell $(i,j)$), or 
    * the sum of live neighbors is 2 *and* cell $(i,j)$ is alive, 
    
  then set `B[i,j] = 1`.

#### Exercise 4

Create a function `my_game_it(A)` that given the current state of the game `A` returns the status of the grid in the next iteration.

**Hint 1:** Since in the original game the grid is infinite, to fit it into an array we should define some *boundary conditions*, i.e., the value of the cells at the edge of our array. Two widely used possibilities (among which you can choose your favorite) are:

* *zero boundary conditions*, i.e., setting all boundary values to zero. This is a bit easier to code but leads to spurious results when the living cells reach the boundary.
* *periodic boundary conditions*: i.e., *gluing* the edges. In this case we must take care of how we define the nearest neighbors at the edges. There you may want to use some `if` statements of the kind:

```julia
if j == 1 
    j_left = N
else
    j_left = j-1
end
```
Or more concisely, using the so called *ternary operator*:
```julia
j == 1 ? j_left = N : j_left = j-1
```
This is the Julia equivalent for the Python expression 
```python 
j = N if j==1 else j-1```

**Hint 2:** Pay attention to the order of the loops! Since Julia is column-major, which loop should go inside to get the maximum speed?

In [ ]:
function my_game_it(A)
    M, N = size(A)
    B = zeros(Int64, M, N)
    
    # Here goes your code
    
    return B
end

# 4.2 CSVs and DataFrames

You can check your implementation by loading the examples in the `game_examples` folder:
* All examples starting with `still` should not change at all.
* All examples starting with `osc` should oscillate with period 2.

In [ ]:
using CSV, DataFrames, Plots
include("GameOfLife.jl")  # import some convenient functions and the reference implementation of the Game

df = CSV.read("game_examples/osc_beacon.csv")

As you can see, `CSV` loads files into `DataFrames`. Cool! Accesing Julia's Dataframes is as simple as in Python, plus we have all of Julia's cool features:

In [ ]:
df.x1

In [ ]:
print.(df.x2); # we can also use broadcasting with DataFrame columns

Anyway, let's come back to our Game of Life. For using `my_game_it` we just have to convert `df` to an array:

In [ ]:
A = Array(df)
B = my_game_it(A)
C = my_game_it(B)

# Plot them
plot1 = plot_it(A); title!("It. 0")
plot2 = plot_it(B); title!("It. 1")
plot3 = plot_it(C); title!("It. 2")
plot(plot1, plot2, plot3,layout = (1,3),size = (900,300))

**Task**: Load other examples and see if they work as they should. 

Finally, running the following cell should yield the gif of the beginning of the notebook

In [ ]:
A = Array(CSV.read("game_examples/intro.csv"))
anim = @animate for i in 1:60
    plot_it(A)
    A .= my_game_it(A)
end
gif(anim,fps=5)

Great! Now, the main motivation to learn Julia was... that it is fast! In the folder `benchmark/` you will find the `game_it` function coded in Python, Matlab and Julia, together with some benchmarking to compare the speeds when running it on a $N\times N$ grid. If you are running the notebooks locally you may also run the benchmarks yourself! 

In any case, you may want to open the file `benchmark/life_julia.jl` and take note of the final lines, because there is an example of how we add columns to DataFrames. 

Well, let's take a look at the resulting Julia's benchmark results:

In [ ]:
CSV.read("benchmark/results_julia.csv")

#### Exercise 5

1. Join the results files in `benchmark/*.csv` into a single DataFrame and save the resulting table to `benchmark/benchmark.csv`. 
2. Read this new file and plot the results. Don't forget to use logscale if needed and add labels and a title!

**Hint**: Be careful! Not always all the data comes in the same format.

In [ ]:
# Here goes your code for joining and saving


# 4.3 Enhancing performance

Ok, so in Julia this implementation runs waaay faster than in Python, and 3 times faster than in Matlab. Not bad! Now, I am pretty sure that your implementation is good, but it's never a bad idea to check Julia's [performance tips](https://docs.julialang.org/en/v1/manual/performance-tips/). Some of the most important are:

* **Avoid global variables**: so far so good (I hope).
* **Measure performance with `@time` and pay attention to memory allocation**: good, let's do it. For the version in the solutions the first time I run it with a 1024x1024 input I obtained the following results:

```0.054303 seconds (49.75 k allocations: 10.554 MiB, 11.94% gc time)```

And the second time,

```0.014615 seconds (6 allocations: 8.000 MiB)```

Let's break this down:

  * **`seconds`** of course is the total time elapsed.
  * **`allocation`** is the act of reserving space in memmory, and **`gc`**, or *garbage collection*, is the freeing of this memory. Unexpectedly high allocations (and specially gc) are usually a sign of some problem in the code, often related to the creation of a high number of temporary arrays.
  
So why are the results so different? Well, the first time `my_game_it` and `@time` are run, they must be compiled, which will consume resources. That's why you shouldn't take the first result seriously. When run a second time, the compiler checks if there exists an already compiled `my_game_it` for the current type of inputs. In case there is, cheers! It only has to run it again, with a different input. The second run of `@time` yields a much better result, which make us confident that our code is probably working nicely.

**Task**: Check if you get similar results with your implementation

In [ ]:
# Here goes your code

Other important performance tips are:

* **Write type-stable code**. Julia performance comes from compiling the code, and the compiler won't have an easy time if the types of the variables are ambiguous or get changed all the time. Think about it this way:
    * If the code for a function is type-stable, most of the decisions regarding types and memory are made at *compile time*, i.e., only the first time that you run it. 
    * However, if the code is not type-stable, decisions must be made at *run time*, happening every time and thus slowing performance.

  For this reason, though it is technically possible to change the type of variables, you shouldn't do it as long as you can avoid it. 

* Likewise, **Arrays should be initialized belonging to some specific type** (`Int64`, `Float64`,...) that its suited for the subsequent operations and not just `Any`, if this is possible.

An incredible macro for checking type-stability is `@code_warntype`. It runs the code and checks the variables for type ambiguities: if a type is completely determined, it prints it in blue; if it is `Any` (this is, it could be anything, slowing performance), it prints it red; and if it is not completely determined but it's not a big deal ---for example, if it must be one of two options--- it writes it yellow. For the version in the solutions the beginning of the report looks like this:

<div class="output_subarea output_text output_stream output_stdout"><pre>Variables
  #self#<span class="ansi-cyan-fg">::Core.Compiler.Const(my_game_it, false)</span>
  A<span class="ansi-cyan-fg">::Array{Int64,2}</span>
  M<span class="ansi-cyan-fg">::Int64</span>
  @_4<span class="ansi-cyan-fg">::Int64</span>
  N<span class="ansi-cyan-fg">::Int64</span>
  j_r<span class="ansi-cyan-fg">::Int64</span>
  j_l<span class="ansi-cyan-fg">::Int64</span>
  i_d<span class="ansi-cyan-fg">::Int64</span>
  i_u<span class="ansi-cyan-fg">::Int64</span>
  B<span class="ansi-cyan-fg">::Array{Int64,2}</span>
  @_11<span class="ansi-yellow-intense-fg ansi-bold">::Union{Nothing, Tuple{Int64,Int64}}</span>
  j<span class="ansi-cyan-fg">::Int64</span>
  @_13<span class="ansi-yellow-intense-fg ansi-bold">::Union{Nothing, Tuple{Int64,Int64}}</span>
  i<span class="ansi-cyan-fg">::Int64</span>
  sum_neigh<span class="ansi-cyan-fg">::Int64</span>
  @_16<span class="ansi-cyan-fg">::Bool</span>
  @_17<span class="ansi-cyan-fg">::Bool</span>
</pre></div>

Great, so everything is pretty much determined except for some temporal variables `@_11`, `@_13`. Apparently Julia is good at handling little `Union`s, so we have nothing to worry about here! (In case you really really want to know what's going on, check [this answer](https://discourse.julialang.org/t/trying-to-understand-output-of-code-warntype/28000/2)). 

**Task**: Check type stability in your implementation 

In [ ]:
# Here goes your code

#### And finally, the most important performance tip: *Don't get obsessed about performance*

Not all the code is time-critical and stressing from the beginning about performance will ruin both your code and your programming experience. 95% of the time you should chill: Julia has your back covered 😉. And anyway, there's always time for refining performance after things are known to work.

For finally relaxing, let's generate a family of ducklings just chilling in their periodic grid:

In [ ]:
A = Array(CSV.read("game_examples/final.csv"));

anim = @animate for i in 1:44
    plot_it(A)
    A .= my_game_it(A)
end
gif(anim,fps = 2)
